In [ ]:
#!/usr/bin/python
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)

In [ ]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    'G://desktop/githubcode&VS_code/tf_emotion_detector/CK+48',
    target_size=(48,48),
    class_mode='categorical'
)

In [ ]:
print(train_generator[3][1])

import cv2 as cv

In [ ]:
cv.imshow('3',train_generator[3][1])

In [ ]:

model = tf.keras.models.Sequential()
       
model.add(tf.keras.layers.Conv2D(32, (1, 1), strides=1, padding='same', input_shape=(48, 48, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, (5, 5), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2048))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1024))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Activation('softmax'))

sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
logdir = os.path.join('callbacks_CK')#for win10
#for mac:
#logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'emtion_model_CK.h5')

callbacks = [
    tf.keras.callbacks.TensorBoard(logdir),
    tf.keras.callbacks.ModelCheckpoint(output_model_file, save_weights_only = False, monitor = 'accuracy', save_best_only = True)
]

history = model.fit(
    train_generator,
    steps_per_epoch=30,
    nb_epoch=15,
    validation_data=None,
    verbose=1,
    callbacks = callbacks
)

In [ ]:
history.history

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)